In [168]:
import pandas as pd
from pathlib import Path
import json
pd.set_option('display.max_rows', 100)

### Setting up the file paths

In [169]:
base_dir = Path().resolve().parent

brands_file = base_dir / 'data' / 'brands.json'
users_file = base_dir / 'data' / 'users.json'
receipts_file = base_dir / 'data' / 'receipts.json'

### Function to read a JSON file into a python object

In [170]:
def read_json_lines(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    return data

In [171]:
brands_data = read_json_lines(brands_file)
users_data = read_json_lines(users_file)
receipts_data = read_json_lines(receipts_file)

### Created function to perform common operations

In [172]:
def flatten_and_normalize(data, record_path=None, meta=None):
    return pd.json_normalize(data, record_path, meta, sep='_')

def reorder_and_rename_columns(df, order, column_mapping):
    df = df[order]
    df = df.rename(columns=column_mapping)
    return df

def convert_to_datetime(df, date_columns):
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], unit='ms')
    return df

### Converting JSON's into Pandas DataFrames for futher analysis and transformations

#### Brands Data

In [173]:
brands_df = flatten_and_normalize(brands_data)

brands_order = [
    '_id_$oid', 'name', 'brandCode', 'barcode', 'category', 'categoryCode', 'topBrand', 'cpg_$id_$oid', 'cpg_$ref' 
]

brands_column_mapping = {
    '_id_$oid': 'brand_id',
    'barcode': 'barcode',
    'category': 'category',
    'categoryCode': 'category_code',
    'cpg_$id_$oid': 'cpg_id',
    'cpg_$ref': 'cpg_ref',
    'name': 'brand_name',
    'topBrand': 'top_brand',
    'brandCode': 'brand_code'
}

brands_df = reorder_and_rename_columns(brands_df, brands_order, brands_column_mapping)
print("No. of records: ",  len(brands_df))
brands_df.head()

No. of records:  1167


,brand_id,brand_name,brand_code,barcode,category,category_code,top_brand,cpg_id,cpg_ref
0,601ac115be37ce2ead437551,test brand @1612366101024,NaN,511111019862,Baking,BAKING,False,601ac114be37ce2ead437550,Cogs
1,601c5460be37ce2ead43755f,Starbucks,STARBUCKS,511111519928,Beverages,BEVERAGES,False,5332f5fbe4b03c9a25efd0ba,Cogs
2,601ac142be37ce2ead43755d,test brand @1612366146176,TEST BRANDCODE @1612366146176,511111819905,Baking,BAKING,False,601ac142be37ce2ead437559,Cogs
3,601ac142be37ce2ead43755a,test brand @1612366146051,TEST BRANDCODE @1612366146051,511111519874,Baking,BAKING,False,601ac142be37ce2ead437559,Cogs
4,601ac142be37ce2ead43755e,test brand @1612366146827,TEST BRANDCODE @1612366146827,511111319917,Candy & Sweets,CANDY_AND_SWEETS,False,5332fa12e4b03c9a25efd1e7,Cogs


#### Users Data

In [174]:
users_df = flatten_and_normalize(users_data)

users_order = [
    '_id_$oid', 'role', 'signUpSource', 'state', 'createdDate_$date', 'active', 'lastLogin_$date' 
]

users_column_mapping = {
    '_id_$oid': 'user_id',
    'active': 'active',
    'createdDate_$date': 'created_date',
    'lastLogin_$date': 'last_login',
    'role': 'role',
    'signUpSource': 'sign_up_source',
    'state': 'state'
}

users_df = reorder_and_rename_columns(users_df, users_order, users_column_mapping)
users_df = convert_to_datetime(users_df, ['created_date', 'last_login'])

print("No. of records: ",  len(users_df))
users_df.head()

No. of records:  495


,user_id,role,sign_up_source,state,created_date,active,last_login
0,5ff1e194b6a9d73a3a9f1052,consumer,Email,WI,2021-01-03 15:24:04.800,True,2021-01-03 15:25:37.857999872
1,5ff1e194b6a9d73a3a9f1052,consumer,Email,WI,2021-01-03 15:24:04.800,True,2021-01-03 15:25:37.857999872
2,5ff1e194b6a9d73a3a9f1052,consumer,Email,WI,2021-01-03 15:24:04.800,True,2021-01-03 15:25:37.857999872
3,5ff1e1eacfcf6c399c274ae6,consumer,Email,WI,2021-01-03 15:25:30.554,True,2021-01-03 15:25:30.596999936
4,5ff1e194b6a9d73a3a9f1052,consumer,Email,WI,2021-01-03 15:24:04.800,True,2021-01-03 15:25:37.857999872


#### Receipts Data

In [175]:
receipts_df = flatten_and_normalize(receipts_data)

receipts_order = [
    '_id_$oid', 'userId', 'bonusPointsEarned', 'bonusPointsEarnedReason', 'createDate_$date', 
    'dateScanned_$date', 'finishedDate_$date', 'modifyDate_$date', 'pointsAwardedDate_$date', 
    'pointsEarned', 'purchaseDate_$date', 'purchasedItemCount', 'rewardsReceiptStatus', 'totalSpent'
]

receipts_column_mapping = {
    '_id_$oid': 'receipt_id',
    'userId': 'user_id',
    'bonusPointsEarned': 'bonus_points_earned',
    'bonusPointsEarnedReason': 'bonus_points_earned_reason',
    'createDate_$date': 'create_date',
    'dateScanned_$date': 'date_scanned',
    'finishedDate_$date': 'finished_date',
    'modifyDate_$date': 'modify_date',
    'pointsAwardedDate_$date': 'points_awarded_date',
    'pointsEarned': 'points_earned',
    'purchaseDate_$date': 'purchase_date',
    'purchasedItemCount': 'purchased_item_count',
    'rewardsReceiptStatus': 'rewards_receipt_status',
    'totalSpent': 'total_spent'
}

receipts_only_df = reorder_and_rename_columns(receipts_df, receipts_order, receipts_column_mapping)

receipts_only_df = convert_to_datetime(receipts_only_df, [
    'create_date', 'date_scanned', 'finished_date', 'modify_date', 'points_awarded_date', 'purchase_date'
])

receipts_only_df.head()

,receipt_id,user_id,bonus_points_earned,bonus_points_earned_reason,create_date,date_scanned,finished_date,modify_date,points_awarded_date,points_earned,purchase_date,purchased_item_count,rewards_receipt_status,total_spent
0,5ff1e1eb0a720f0523000575,5ff1e1eacfcf6c399c274ae6,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,FINISHED,26.00
1,5ff1e1bb0a720f052300056b,5ff1e194b6a9d73a3a9f1052,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,FINISHED,11.00
2,5ff1e1f10a720f052300057a,5ff1e1f1cfcf6c399c274b0b,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,5,2021-01-03 00:00:00,1.0,REJECTED,10.00
3,5ff1e1ee0a7214ada100056f,5ff1e1eacfcf6c399c274ae6,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4.0,FINISHED,28.00
4,5ff1e1d20a7214ada1000561,5ff1e194b6a9d73a3a9f1052,5.0,All-receipts receipt bonus,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,2.0,FINISHED,1.00


### Using the rewardsReceiptItemList to create a seperate df for the items in each receipt to enable access to the granular data

In [176]:
for receipt in receipts_data:
    receipt['id'] = receipt['_id']['$oid']
    if 'rewardsReceiptItemList' not in receipt:
        receipt['rewardsReceiptItemList'] = []
    for item in receipt['rewardsReceiptItemList']:
        item['receipt_id'] = receipt['id']
        
# Normalize the data to extract rewardsReceiptItemList and associated _id
items_df = flatten_and_normalize(
    receipts_data,
    record_path=['rewardsReceiptItemList'],
    meta=['id']
)

items_df.drop('id', inplace=True, axis=1)
items_df.head()

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [177]:
items_df.columns

Index(['barcode', 'description', 'finalPrice', 'itemPrice', 'needsFetchReview',
       'partnerItemId', 'preventTargetGapPoints', 'quantityPurchased',
       'userFlaggedBarcode', 'userFlaggedNewItem', 'userFlaggedPrice',
       'userFlaggedQuantity', 'receipt_id', 'needsFetchReviewReason',
       'pointsNotAwardedReason', 'pointsPayerId', 'rewardsGroup',
       'rewardsProductPartnerId', 'userFlaggedDescription',
       'originalMetaBriteBarcode', 'originalMetaBriteDescription', 'brandCode',
       'competitorRewardsGroup', 'discountedItemPrice',
       'originalReceiptItemText', 'itemNumber',
       'originalMetaBriteQuantityPurchased', 'pointsEarned', 'targetPrice',
       'competitiveProduct', 'originalFinalPrice',
       'originalMetaBriteItemPrice', 'deleted', 'priceAfterCoupon',
       'metabriteCampaignId'],
      dtype='object')

## Data Quality Checks:

Based on the data I have previewed so far, I have carried out relevant data quality checks based on intuition. These checks can be further expanded and refined based on business logic and other expected characteristics

### User data checks

For the Users data the main goal was to ensure intergrity of the data and check for duplication

In [178]:
users_df.isna().sum()

user_id            0
role               0
sign_up_source    48
state             56
created_date       0
active             0
last_login        62
dtype: int64

In [179]:
print(users_df.dtypes)

user_id                   object
role                      object
sign_up_source            object
state                     object
created_date      datetime64[ns]
active                      bool
last_login        datetime64[ns]
dtype: object


#### Checking for duplicate records
Based on the checks below there seems to be a significant amount of duplication in the user_id column which is important in a Relational DB to conform to the Consistency property in ACID.


In [180]:
user_duplicate_record = users_df[users_df.duplicated('user_id')]

print("Duplicated Records:")
user_duplicate_record.head(70)

Duplicated Records:


,user_id,role,sign_up_source,state,created_date,active,last_login
1,5ff1e194b6a9d73a3a9f1052,consumer,Email,WI,2021-01-03 15:24:04.800,True,2021-01-03 15:25:37.857999872
2,5ff1e194b6a9d73a3a9f1052,consumer,Email,WI,2021-01-03 15:24:04.800,True,2021-01-03 15:25:37.857999872
4,5ff1e194b6a9d73a3a9f1052,consumer,Email,WI,2021-01-03 15:24:04.800,True,2021-01-03 15:25:37.857999872
5,5ff1e194b6a9d73a3a9f1052,consumer,Email,WI,2021-01-03 15:24:04.800,True,2021-01-03 15:25:37.857999872
8,5ff1e194b6a9d73a3a9f1052,consumer,Email,WI,2021-01-03 15:24:04.800,True,2021-01-03 15:25:37.857999872
10,5ff1e194b6a9d73a3a9f1052,consumer,Email,WI,2021-01-03 15:24:04.800,True,2021-01-03 15:25:37.857999872
11,5ff1e1eacfcf6c399c274ae6,consumer,Email,WI,2021-01-03 15:25:30.554,True,2021-01-03 15:25:30.596999936
12,5ff1e194b6a9d73a3a9f1052,consumer,Email,WI,2021-01-03 15:24:04.800,True,2021-01-03 15:25:37.857999872
13,5ff1e1eacfcf6c399c274ae6,consumer,Email,WI,2021-01-03 15:25:30.554,True,2021-01-03 15:25:30.596999936
14,5ff1e194b6a9d73a3a9f1052,consumer,Email,WI,2021-01-03 15:24:04.800,True,2021-01-03 15:25:37.857999872


#### Checking count of each duplicated id

In [181]:
user_duplicate_counts = users_df.groupby('user_id').size().reset_index(name='counts')

user_duplicate_counts = user_duplicate_counts[user_duplicate_counts['counts'] > 1]

user_duplicate_counts['duplicate_records'] = user_duplicate_counts['counts'] - 1

user_duplicate_counts[['user_id', 'duplicate_records']]


,user_id,duplicate_records
0,54943462e4b07e684157a532,19
3,59c124bae4b0299e55b0f330,17
4,5a43c08fe4b014fd6b6a0612,7
8,5fa41775898c7a11a6bcef3e,17
9,5fb0a078be5fc9775c1f3945,1
10,5fbc35711d967d1222cbfefc,2
11,5fc961c3b8cfca11a077dd33,19
12,5ff1e194b6a9d73a3a9f1052,10
17,5ff1e1eacfcf6c399c274ae6,3
19,5ff36a3862fde912123a4460,4


In [106]:
user_id_to_check = '5a43c08fe4b014fd6b6a0612'

# Extract JSON data for the specific user_id
json_data_for_user = [user for user in users_data if user['_id']['$oid'] == user_id_to_check]

print("JSON Data for user_id:", user_id_to_check)
print(json.dumps(json_data_for_user, indent=4))

df_data_for_user = users_df[users_df['user_id'] == user_id_to_check]

print("DataFrame Data for user_id:", user_id_to_check)
df_data_for_user

JSON Data for user_id: 5a43c08fe4b014fd6b6a0612
[
    {
        "_id": {
            "$oid": "5a43c08fe4b014fd6b6a0612"
        },
        "active": true,
        "createdDate": {
            "$date": 1514389647059
        },
        "lastLogin": {
            "$date": 1613146957155
        },
        "role": "consumer"
    },
    {
        "_id": {
            "$oid": "5a43c08fe4b014fd6b6a0612"
        },
        "active": true,
        "createdDate": {
            "$date": 1514389647059
        },
        "lastLogin": {
            "$date": 1613146957155
        },
        "role": "consumer"
    },
    {
        "_id": {
            "$oid": "5a43c08fe4b014fd6b6a0612"
        },
        "active": true,
        "createdDate": {
            "$date": 1514389647059
        },
        "lastLogin": {
            "$date": 1613146957155
        },
        "role": "consumer"
    },
    {
        "_id": {
            "$oid": "5a43c08fe4b014fd6b6a0612"
        },
        "active": true,
        

,user_id,role,sign_up_source,state,created_date,active,last_login
422,5a43c08fe4b014fd6b6a0612,consumer,NaN,NaN,2017-12-27 15:47:27.059,True,2021-02-12 16:22:37.155000064
423,5a43c08fe4b014fd6b6a0612,consumer,NaN,NaN,2017-12-27 15:47:27.059,True,2021-02-12 16:22:37.155000064
424,5a43c08fe4b014fd6b6a0612,consumer,NaN,NaN,2017-12-27 15:47:27.059,True,2021-02-12 16:22:37.155000064
425,5a43c08fe4b014fd6b6a0612,consumer,NaN,NaN,2017-12-27 15:47:27.059,True,2021-02-12 16:22:37.155000064
426,5a43c08fe4b014fd6b6a0612,consumer,NaN,NaN,2017-12-27 15:47:27.059,True,2021-02-12 16:22:37.155000064
428,5a43c08fe4b014fd6b6a0612,consumer,NaN,NaN,2017-12-27 15:47:27.059,True,2021-02-12 16:22:37.155000064
430,5a43c08fe4b014fd6b6a0612,consumer,NaN,NaN,2017-12-27 15:47:27.059,True,2021-02-12 16:22:37.155000064
431,5a43c08fe4b014fd6b6a0612,consumer,NaN,NaN,2017-12-27 15:47:27.059,True,2021-02-12 16:22:37.155000064


#### Group by 'user_id' and check if all the rows are the same

In [183]:

grouped = user_duplicate_record.groupby('user_id').apply(lambda x: x.nunique() == 1)

grouped

,user_id,role,sign_up_source,state,created_date,active,last_login
user_id,,,,,,,
54943462e4b07e684157a532,True,True,False,False,True,True,True
59c124bae4b0299e55b0f330,True,True,False,True,True,True,True
5a43c08fe4b014fd6b6a0612,True,True,False,False,True,True,True
5fa41775898c7a11a6bcef3e,True,True,True,False,True,True,True
5fb0a078be5fc9775c1f3945,True,True,True,True,True,True,False
5fbc35711d967d1222cbfefc,True,True,True,False,True,True,True
5fc961c3b8cfca11a077dd33,True,True,True,True,True,True,True
5ff1e194b6a9d73a3a9f1052,True,True,True,True,True,True,True
5ff1e1eacfcf6c399c274ae6,True,True,True,True,True,True,True


Based on the above result, majority of the rows have the same data duplicated, ideally these can be deduplicated while loading the data into the Database. For the rows that have differening values, they would require further analysis and appropriate business logic to handle them.

#### Checking for unique values in the 'roles' column

According to the data dictionary the values should be by default 'consumer' but given the presence of the values 'fetch-staff', there needs to be rules set in place on how to capture those results. 

In [157]:
users_df['role'].unique()

array(['consumer', 'fetch-staff'], dtype=object)

#### Checking for values in the 'active' column

In [107]:
users_df['active'].unique()

array([ True, False])

### Brand Table data check

In [108]:
brands_df.isna().sum()

brand_id           0
brand_name         0
brand_code       234
barcode            0
category         155
category_code    650
top_brand        612
cpg_id             0
cpg_ref            0
dtype: int64

In [133]:
print(brands_df.dtypes)

brand_id         object
brand_name       object
brand_code       object
barcode          object
category         object
category_code    object
top_brand        object
cpg_id           object
cpg_ref          object
dtype: object


#### Checking for duplicate brand_id

In [186]:
brand_duplicate_id_counts = brands_df[brands_df.duplicated('brand_id')]

print("Duplicated Records:")
brand_duplicate_id_counts.head(70)

Duplicated Records:


,brand_id,brand_name,brand_code,barcode,category,category_code,top_brand,cpg_id,cpg_ref


#### Checking for duplicate brand_code

In [110]:
brand_duplicate_record = brands_df[brands_df.duplicated('brand_code')]

print("Duplicated Records:")
brand_duplicate_record.head(70)

Duplicated Records:


,brand_id,brand_name,brand_code,barcode,category,category_code,top_brand,cpg_id,cpg_ref
11,57c08106e4b0718ff5fcb02c,MorningStar,NaN,511111102540,NaN,NaN,NaN,5332f5f2e4b03c9a25efd0aa,Cpgs
18,5fb28549be37ce522e165cb5,test brand @1605535049181,NaN,511111317364,Baking,BAKING,False,5fb28549be37ce522e165cb4,Cogs
23,5332f5fee4b03c9a25efd0bd,Bottled Starbucks,NaN,511111303947,NaN,NaN,NaN,53e10d6368abd3c7065097cc,Cpgs
24,5332fa7ce4b03c9a25efd22e,Full Throttle,NaN,511111802914,NaN,NaN,NaN,5332f5ebe4b03c9a25efd0a8,Cpgs
25,5e9f18bfbe37ce3e45b6a77f,PopUp Brand A,NaN,511111914549,Baking,BAKING,NaN,5e9f12f5be37ce3e45b6a77e,Cogs
29,5f4936ddbe37ce52f8314fd9,test brand @1598633693011,NaN,511111315957,Baking,BAKING,NaN,5f4936dcbe37ce52f8314fd8,Cogs
31,5fd2a0aebe37ce49eb72c0ee,test brand @1607639214411,NaN,511111518112,Baking,BAKING,False,5fd2a0aebe37ce49eb72c0ed,Cogs
34,5332f772e4b03c9a25efd125,Gold Medal,NaN,511111103653,NaN,NaN,NaN,5332f5f3e4b03c9a25efd0ae,Cpgs
43,5332f765e4b03c9a25efd11f,Glaceau vitaminwater,NaN,511111503699,NaN,NaN,NaN,5332f5ebe4b03c9a25efd0a8,Cpgs
46,5332f5f3e4b03c9a25efd0ad,Gree Giant,NaN,511111104025,NaN,NaN,NaN,5332f5f3e4b03c9a25efd0ae,Cpgs


In [111]:
brand_duplicate_counts = brands_df.groupby('brand_code').size().reset_index(name='counts')

brand_duplicate_counts = brand_duplicate_counts[brand_duplicate_counts['counts'] > 1]

brand_duplicate_counts['duplicate_records'] = brand_duplicate_counts['counts'] - 1

brand_duplicate_counts[['brand_code', 'duplicate_records']]


,brand_code,duplicate_records
0,,34
254,GOODNITES,1
277,HUGGIES,1


Based on the analysis here, the duplications seem to be majority of 'null' and couple of other brands. On scaling, when we would potentially be adding more brands or when probably other processes populate the values for the nulls we would be in a better position to create data checks. For now it would be best to have them logged for monitoring purposes.

#### Exploring cpg_id(For data modelling)

In [112]:
brand_duplicate_counts = brands_df.groupby('cpg_id').size().reset_index(name='counts')

brand_duplicate_counts = brand_duplicate_counts[brand_duplicate_counts['counts'] > 1]

brand_duplicate_counts['duplicate_records'] = brand_duplicate_counts['counts'] - 1

brand_duplicate_counts[['cpg_id', 'duplicate_records']]

,cpg_id,duplicate_records
0,5332f5ebe4b03c9a25efd0a8,33
1,5332f5f2e4b03c9a25efd0aa,24
3,5332f5f3e4b03c9a25efd0ae,50
4,5332f5f6e4b03c9a25efd0b4,36
5,5332f5fbe4b03c9a25efd0ba,84
7,5332f709e4b03c9a25efd0f1,57
10,5332f7a7e4b03c9a25efd134,26
11,5332f7ffe4b03c9a25efd16b,5
12,5332fa12e4b03c9a25efd1e7,74
13,5332fa58e4b03c9a25efd215,3


In [113]:
cpg_id_to_check = '5332f5ebe4b03c9a25efd0a8'

json_data_for_cpg_id = [brand for brand in brands_data if 'cpg' in brand and brand['cpg']['$id']['$oid'] 
                        == cpg_id_to_check]

print("JSON Data for cpg_id:", cpg_id_to_check)
print(json.dumps(json_data_for_cpg_id, indent=4))

df_data_for_cpg_id = brands_df[brands_df['cpg_id'] == cpg_id_to_check]

print("DataFrame Data for cpg_id:", cpg_id_to_check)
df_data_for_cpg_id

JSON Data for cpg_id: 5332f5ebe4b03c9a25efd0a8
[
    {
        "_id": {
            "$oid": "5332fa7ce4b03c9a25efd22e"
        },
        "name": "Full Throttle",
        "cpg": {
            "$ref": "Cpgs",
            "$id": {
                "$oid": "5332f5ebe4b03c9a25efd0a8"
            }
        },
        "barcode": "511111802914"
    },
    {
        "_id": {
            "$oid": "5332f765e4b03c9a25efd11f"
        },
        "name": "Glaceau vitaminwater",
        "cpg": {
            "$ref": "Cpgs",
            "$id": {
                "$oid": "5332f5ebe4b03c9a25efd0a8"
            }
        },
        "barcode": "511111503699"
    },
    {
        "_id": {
            "$oid": "5332fa7ee4b03c9a25efd230"
        },
        "name": "Powerade",
        "cpg": {
            "$ref": "Cpgs",
            "$id": {
                "$oid": "5332f5ebe4b03c9a25efd0a8"
            }
        },
        "barcode": "511111702894"
    },
    {
        "_id": {
            "$oid": "5332f760e4b03c

,brand_id,brand_name,brand_code,barcode,category,category_code,top_brand,cpg_id,cpg_ref
24,5332fa7ce4b03c9a25efd22e,Full Throttle,NaN,511111802914,NaN,NaN,NaN,5332f5ebe4b03c9a25efd0a8,Cpgs
43,5332f765e4b03c9a25efd11f,Glaceau vitaminwater,NaN,511111503699,NaN,NaN,NaN,5332f5ebe4b03c9a25efd0a8,Cpgs
77,5332fa7ee4b03c9a25efd230,Powerade,NaN,511111702894,NaN,NaN,NaN,5332f5ebe4b03c9a25efd0a8,Cpgs
147,5332f760e4b03c9a25efd11b,Honest Ade,NaN,511111603719,NaN,NaN,NaN,5332f5ebe4b03c9a25efd0a8,Cpgs
195,5332fa7ae4b03c9a25efd229,Fanta,NaN,511111402961,NaN,NaN,NaN,5332f5ebe4b03c9a25efd0a8,Cpgs
227,5332fa7be4b03c9a25efd22b,Diet Cherry Coke,NaN,511111702948,NaN,NaN,NaN,5332f5ebe4b03c9a25efd0a8,Cpgs
272,5332fa79e4b03c9a25efd226,Mello Yello,NaN,511111602996,NaN,NaN,NaN,5332f5ebe4b03c9a25efd0a8,Cpgs
290,5332fa73e4b03c9a25efd21c,Diet Coke,NaN,511111703051,NaN,NaN,NaN,5332f5ebe4b03c9a25efd0a8,Cpgs
350,5332f753e4b03c9a25efd10f,NOS,NaN,511111703778,NaN,NaN,NaN,5332f5ebe4b03c9a25efd0a8,Cpgs
387,5332f760e4b03c9a25efd11a,Honest Tea,NaN,511111803720,NaN,NaN,NaN,5332f5ebe4b03c9a25efd0a8,Cpgs


Seems like cpg_id corresponds roughly to 'catgeory' of the product and could be usefull for any future requirements from the stakeholders. A cpg table would be a good idea for expansion.

### Reciepts data checks

In [118]:
receipts_only_df.isna().sum()

receipt_id                      0
user_id                         0
bonus_points_earned           575
bonus_points_earned_reason    575
create_date                     0
date_scanned                    0
finished_date                 551
modify_date                     0
points_awarded_date           582
points_earned                 510
purchase_date                 448
purchased_item_count          484
rewards_receipt_status          0
total_spent                   435
dtype: int64

In [134]:
print(receipts_only_df.dtypes)

receipt_id                            object
user_id                               object
bonus_points_earned                  float64
bonus_points_earned_reason            object
create_date                   datetime64[ns]
date_scanned                  datetime64[ns]
finished_date                 datetime64[ns]
modify_date                   datetime64[ns]
points_awarded_date           datetime64[ns]
points_earned                         object
purchase_date                 datetime64[ns]
purchased_item_count                 float64
rewards_receipt_status                object
total_spent                           object
dtype: object


#### Checking for duplicate receipt_id's

In [121]:
receipts_duplicate_record = receipts_only_df[receipts_only_df.duplicated('receipt_id')]

print("Duplicated Records:")
receipts_duplicate_record.head(70)

Duplicated Records:


,receipt_id,user_id,bonus_points_earned,bonus_points_earned_reason,create_date,date_scanned,finished_date,modify_date,points_awarded_date,points_earned,purchase_date,purchased_item_count,rewards_receipt_status,total_spent


#### Checking for rewards status

In [122]:
receipts_only_df['rewards_receipt_status'].unique()

array(['FINISHED', 'REJECTED', 'FLAGGED', 'SUBMITTED', 'PENDING'],
      dtype=object)

#### Checking purchase_date is earlier than create_date

In [159]:
purchase_date_anamoly_df = receipts_only_df[(receipts_only_df['purchase_date'] > receipts_only_df['create_date'])]

print("Rows where create_date is older than purchase_date:")
purchase_date_anamoly_df[['receipt_id', 'purchase_date', 'create_date']]

Rows where create_date is older than purchase_date:


,receipt_id,purchase_date,create_date
12,5ff1e1b60a7214ada100055c,2021-02-03 15:24:38,2021-01-03 15:24:38
14,5ff1e1b20a7214ada100055a,2021-02-03 15:24:35,2021-01-03 15:24:34
85,5ff4ce640a7214ada10005e0,2021-02-05 20:39:00,2021-01-05 20:39:00
139,5ff73be10a7214ada1000619,2021-02-07 16:50:41,2021-01-07 16:50:41
158,5ff873f10a720f052300064f,2021-02-08 15:02:10,2021-01-08 15:02:09
190,5ffcb4900a720f0515000002,2021-02-11 20:26:56,2021-01-11 20:26:56
244,5fff26ee0a720f05f300001a,2021-02-13 16:59:26,2021-01-13 16:59:26
265,5fff26f10a7214ad4c000018,2021-02-13 16:59:29,2021-01-13 16:59:29
294,6000d4bc0a7214ad4c000070,2021-02-14 23:33:17,2021-01-14 23:33:16
362,600887560a720f05fa000098,2021-02-20 19:41:10,2021-01-20 19:41:10


In [128]:
len(purchase_date_anamoly_df)

13

Given that there are 13 cases where the purchase date is after the create date there needs to be checks in place to handle this or if there is a business logic that explains this anamoly

### Receipt items data check

In [99]:
items_df.isna().sum()

barcode                               3851
description                            381
finalPrice                             174
itemPrice                              174
needsFetchReview                      6128
partnerItemId                            0
preventTargetGapPoints                6583
quantityPurchased                      174
userFlaggedBarcode                    6604
userFlaggedNewItem                    6618
userFlaggedPrice                      6642
userFlaggedQuantity                   6642
receipt_id                               0
needsFetchReviewReason                6722
pointsNotAwardedReason                6601
pointsPayerId                         5674
rewardsGroup                          5210
rewardsProductPartnerId               4672
userFlaggedDescription                6736
originalMetaBriteBarcode              6870
originalMetaBriteDescription          6931
brandCode                             4341
competitorRewardsGroup                6666
discountedI

In [135]:
print(items_df.dtypes)

barcode                                object
description                            object
finalPrice                             object
itemPrice                              object
needsFetchReview                       object
partnerItemId                          object
preventTargetGapPoints                 object
quantityPurchased                     float64
userFlaggedBarcode                     object
userFlaggedNewItem                     object
userFlaggedPrice                       object
userFlaggedQuantity                   float64
receipt_id                             object
needsFetchReviewReason                 object
pointsNotAwardedReason                 object
pointsPayerId                          object
rewardsGroup                           object
rewardsProductPartnerId                object
userFlaggedDescription                 object
originalMetaBriteBarcode               object
originalMetaBriteDescription           object
brandCode                         

#### Updating the data types for numerical columns

In [150]:
items_df['finalPrice'] = items_df['finalPrice'].astype('float')
items_df['itemPrice'] = items_df['itemPrice'].astype('float')
items_df['userFlaggedQuantity'] = items_df['userFlaggedQuantity'].astype('float')
items_df['originalMetaBriteQuantityPurchased'] = items_df['originalMetaBriteQuantityPurchased'].astype('float')
items_df['pointsEarned'] = items_df['pointsEarned'].astype('float')
items_df['targetPrice'] = items_df['targetPrice'].astype('float')
items_df['originalFinalPrice'] = items_df['originalFinalPrice'].astype('float')
items_df['originalMetaBriteItemPrice'] = items_df['originalMetaBriteItemPrice'].astype('float')
items_df['priceAfterCoupon'] = items_df['priceAfterCoupon'].astype('float')

In [152]:
print(items_df.dtypes)

barcode                                object
description                            object
finalPrice                            float64
itemPrice                             float64
needsFetchReview                       object
partnerItemId                          object
preventTargetGapPoints                 object
quantityPurchased                     float64
userFlaggedBarcode                     object
userFlaggedNewItem                     object
userFlaggedPrice                       object
userFlaggedQuantity                   float64
receipt_id                             object
needsFetchReviewReason                 object
pointsNotAwardedReason                 object
pointsPayerId                          object
rewardsGroup                           object
rewardsProductPartnerId                object
userFlaggedDescription                 object
originalMetaBriteBarcode               object
originalMetaBriteDescription           object
brandCode                         

#### Checking if the columns 'quantityPurchased' and 'finalPrice' are negative

In [160]:
invalid_items_df = items_df[(items_df['quantityPurchased'] < 0) & (items_df['finalPrice'] < 0)]

print("Rows with negative quantityPurchased and finalPrice:")
invalid_items_df[['receipt_id', 'quantityPurchased', 'finalPrice']].head(70)

Rows with negative quantityPurchased and finalPrice:


,receipt_id,quantityPurchased,finalPrice


#### Find common barcodes between items and brands dataframes

In [96]:
items_barcode = set(items_df['barcode'])
brands_barcode = set(brands_df['barcode'])

known_brands = items_barcode.intersection(brands_barcode)
known_brands

{'511111001485',
 '511111001768',
 '511111003960',
 '511111004127',
 '511111101451',
 '511111104186',
 '511111104537',
 '511111204206',
 '511111502142',
 '511111518044',
 '511111602118',
 '511111704140',
 '511111802358',
 '511111901587',
 '511111902690',
 '511111904175'}

In [97]:
len(known_brands)

16

#### Find common brand codes between items and brands dataframes

In [100]:
items_brandcode = set(items_df['brandCode'])
brands_barcode = set(brands_df['brand_code'])

known_brands_codes = items_brandcode.intersection(brands_barcode)
known_brands_codes

{'ARNOLD',
 'CHEETOS',
 'CLASSICO',
 'COOL WHIP',
 'COTTONELLE',
 'CRACKER BARREL',
 'DOLE CHILLED FRUIT JUICES',
 'DORITOS',
 'FINISH',
 'GREY POUPON',
 "HELLMANN'S/BEST FOODS",
 'HUGGIES',
 'JELL-O',
 'JUST CRACK AN EGG',
 'KETTLE BRAND',
 'KLEENEX',
 'KLONDIKE',
 'KNORR',
 'KRAFT',
 'LUNCHABLES',
 'MOUNTAIN DEW',
 'NATURE VALLEY',
 'ORE-IDA',
 'OSCAR MAYER',
 'PACIFIC FOODS',
 'PEPPERIDGE FARM',
 'PEPSI',
 'PHILADELPHIA',
 'PLANTERS',
 'PREGO',
 'QUAKER',
 'RICE-A-RONI',
 'SARGENTO',
 'STOVE TOP',
 'SWANSON',
 'TACO BELL',
 'TOSTITOS',
 'V8',
 'VELVEETA',
 'VIVA',
 'YUBAN',
 nan}

In [101]:
len(known_brands_codes)

42